# Champions general statistics

Get the saved games data

In [1]:
import json, os
gamesRawData = []
for i in os.listdir("./games"):
    if i.endswith(".json"):
        with open("./games/"+i,"r") as f:
            gamesRawData.append(json.loads(f.read()))

Initialize the champion stats data structure.

In [2]:
import requests

response = requests.get('http://ddragon.leagueoflegends.com/cdn/8.19.1/data/en_US/champion.json')
champonRawData = json.loads(response.text)

championsStats = {}
for key,champion in champonRawData['data'].items():
    championsStats[int(champion['key'])] = {
                    "champion":champion['name'],
                    "games":0,
                    "gamesPlayed":0,
                    "wins":0,
                    "teamKills":0,
                    "kills":0,
                    "deaths":0,
                    "assists":0,
                    "bans":0
                }


Parse all the games to get statistics by champion

In [3]:
for game in gamesRawData:
    #Count the champion bans
    for t in game["teams"]:
        for b in t["bans"]:
            championsStats[b["championId"]]["bans"] += 1
            championsStats[b["championId"]]["games"]+=1
    
    #Count the kills of the team to later get the kill participation
    teamKills = {100:0,200:0}
    for p in game["participants"]:
        teamKills[p["teamId"]] += p["stats"]["kills"]
    
    #Count the wins and basic stats like kills, deaths and assits
    for p in game["participants"]:
        championsStats[p["championId"]]["games"]+=1
        championsStats[p["championId"]]["gamesPlayed"]+=1
        if p["stats"]["win"]:
            championsStats[p["championId"]]["wins"] += 1
        
        championsStats[p["championId"]]["teamKills"] += teamKills[p["teamId"]]
        championsStats[p["championId"]]["kills"] += p["stats"]["kills"]
        championsStats[p["championId"]]["deaths"] += p["stats"]["deaths"]
        championsStats[p["championId"]]["assists"] += p["stats"]["assists"]

We make a difference between games and gamesPlayed where games counts every game where the champion has been either played or banned.

Use pandas Dataframe to manipulate the data

In [4]:
import pandas as pd, numpy as np
df = pd.DataFrame(championsStats).T

#Get the winrate for each champion
#wr is the raw winrate
df["wr"] = 0
df.loc[df["gamesPlayed"]>0,"wr"] = df.loc[df["gamesPlayed"]>0]["wins"]/df.loc[df["gamesPlayed"]>0]["gamesPlayed"]
#winrate is the beautified winrate
df["winrate"] = (df["wr"] * 100).map('{:,.2f}'.format).apply(lambda s: str(s)+"%")

#Get the KDA
df["KDA"] = 0
df.loc[df["deaths"]>0,"KDA"] = ((df.loc[df["deaths"]>0]["kills"] + df.loc[df["deaths"]>0]["assists"]) / df.loc[df["deaths"]>0]["deaths"])
#Handling the case when KDA is infinite (deaths=0)
df.loc[df["deaths"]==0,"KDA"]=(df.loc[df["deaths"]==0]["kills"] + df.loc[df["deaths"]==0]["assists"])
#Beautify KDA
df["KDA"] = df["KDA"].map('{:,.2f}'.format)

#Get kill participation and beautify it
df["KP"] = 0
df.loc[df["teamKills"]>0,"KP"] = (((df.loc[df["teamKills"]>0]["kills"] + df.loc[df["teamKills"]>0]["assists"]) / df.loc[df["teamKills"]>0]["teamKills"]) * 100).map('{:,.2f}'.format).apply(lambda s: str(s)+"%")

Resulting Dataframe

In [5]:
df

,assists,bans,champion,deaths,games,gamesPlayed,kills,teamKills,wins,wr,winrate,KDA,KP
1,0,0,Annie,0,0,0,0,0,0,0,0.00%,0.00,0
2,125,29,Olaf,53,54,25,73,281,10,0.4,40.00%,3.74,70.46%
3,116,26,Galio,55,52,26,61,266,12,0.461538,46.15%,3.22,66.54%
4,0,0,Twisted Fate,0,0,0,0,0,0,0,0.00%,0.00,0
5,149,19,Xin Zhao,62,46,27,71,338,16,0.592593,59.26%,3.55,65.09%
6,182,64,Urgot,99,108,44,133,526,26,0.590909,59.09%,3.18,59.89%
7,112,43,LeBlanc,49,67,24,90,321,15,0.625,62.50%,4.12,62.93%
8,0,0,Vladimir,0,0,0,0,0,0,0,0.00%,0.00,0
9,0,0,Fiddlesticks,0,0,0,0,0,0,0,0.00%,0.00,0
10,0,0,Kayle,0,0,0,0,0,0,0,0.00%,0.00,0


Exporting the data to csv

In [6]:
df.index.name='championId'
df.to_csv("champion.csv")

Some manipulation examples

In [7]:
(df[["champion","kills","deaths","assists","KDA","teamKills","KP","wins","gamesPlayed","games","bans","winrate","wr"]] #Selecting the columns to show and their order
    .sort_values(["wr","gamesPlayed"], ascending=False) #Sorting champions by winrate and games played
    .drop("wr", axis=1)) #Drop the "wr" column which is only here for sorting purpose, the "winrate" column giving the same information but beautified

,champion,kills,deaths,assists,KDA,teamKills,KP,wins,gamesPlayed,games,bans,winrate
championId,,,,,,,,,,,,
27,Singed,7,3,14,7.00,35,60.00%,2,2,4,2,100.00%
43,Karma,3,2,16,9.50,24,79.17%,2,2,3,1,100.00%
154,Zac,3,1,21,24.00,31,77.42%,2,2,4,2,100.00%
157,Yasuo,2,6,21,3.83,28,82.14%,2,2,7,5,100.00%
24,Jax,2,1,4,6.00,12,50.00%,1,1,2,1,100.00%
34,Anivia,3,0,12,15.00,18,83.33%,1,1,5,4,100.00%
120,Hecarim,5,2,3,4.00,13,61.54%,1,1,3,2,100.00%
122,Darius,1,1,5,6.00,11,54.55%,1,1,5,4,100.00%
240,Kled,2,1,7,9.00,20,45.00%,1,1,1,0,100.00%


In [8]:
#Getting percentage of champions either played or banned in Worlds 2018
df[df["games"] > 0].shape[0] / df.shape[0]

0.6453900709219859

In [9]:
(df[["champion","kills","deaths","assists","KDA","teamKills","KP","wins","gamesPlayed","games","bans","winrate","wr"]] #Selecting the columns to show and their order
    .sort_values(["games"], ascending=False) #Sorting champions by games (played and banned)
    .drop("wr", axis=1)) #Drop the "wr" column which is only here for sorting purpose, the "winrate" column giving the same information but beautified

,champion,kills,deaths,assists,KDA,teamKills,KP,wins,gamesPlayed,games,bans,winrate
championId,,,,,,,,,,,,
266,Aatrox,74,83,135,2.52,400,52.25%,17,36,109,73,47.22%
6,Urgot,133,99,182,3.18,526,59.89%,26,44,108,64,59.09%
12,Alistar,27,110,301,2.98,476,68.91%,25,45,104,59,55.56%
145,Kai'Sa,285,112,257,4.84,782,69.31%,36,66,89,23,54.55%
84,Akali,81,38,74,4.08,231,67.10%,10,18,86,68,55.56%
497,Rakan,37,96,331,3.83,464,79.31%,20,40,81,41,50.00%
223,Tahm Kench,16,63,168,2.92,298,61.74%,13,28,79,51,46.43%
39,Irelia,98,79,128,2.86,355,63.66%,18,26,78,52,69.23%
498,Xayah,157,79,194,4.44,510,68.82%,21,42,71,29,50.00%
